<a href="https://colab.research.google.com/github/manumeehl/trajectories/blob/main/trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install tesseract-ocr
!pip install opencv-python-headless pytesseract pytube

In [ ]:
from google.colab.patches import cv2_imshow as show
from pytube import YouTube
import pytesseract
import cv2
import re

In [19]:
def get_video(url, path):
  vid = YouTube(url)
  print(f"Downloading Video '{vid.streams[0].title}'")
  vid.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=path, filename='launch.mp4')
  print('Download complete')

In [ ]:
def custom_crop(frame):
  height, width, _ = frame.shape
  return frame[height-140:height,0:1920]

In [76]:
def clock_to_secs(clockstring):
  ''' Convert a mission clock string to an integer second value '''
  tense = -1 if clockstring[:2] == 't-' else 1
  # Remove the T+- sign
  clockstring = clockstring [2:]
  # Get the seconds
  units = clockstring.split(':')
  hours = int(units[0])
  minutes = int(units[1])
  seconds = int(units[2])

  return hours * 3600 + minutes * 60 + seconds

In [78]:
def get_props(string):
  ''' Extract mission clock, altitude and velocity from a given string '''
  string = string.lower()

  # Apply a blacklist
  blacklist = ['stage 1', 'stage 2', 'falcon 9']
  for w in blacklist:
    string = string.replace(w, '')

  # Find all numbers and floats in the string
  numbers = re.findall(r'\d+(?:\.\d+)?', string)

  # Find the mission clock (T+00:00:00)
  mission_clock = re.findall(r't[+-]\d{2}:\d{2}:\d{2}', string)

  if mission_clock:
    mission_clock = mission_clock[0]

    # Remove numbers which also occur in the mission clock
    mission_clock_numbers = re.findall(r'\d+(?:\.\d+)?', ' '.join(mission_clock))
    for num in mission_clock_numbers:
      if num in numbers:
        numbers.remove(num)

    # Calculate the elapsed amount of seconds
    elapsed = clock_to_secs(mission_clock)
  else:
    mission_clock = ''
    elapsed = 0

  # Remove duplicates from the numbers list
  numbers = list(set(numbers))

  altitude, velocity = 0, 0
  if numbers:
    altitude = min(numbers)
    velocity = max(numbers)

  # Return the sorted numbers and mission clock
  return {
      'clock': mission_clock,
      'seconds': elapsed,
      'velocity': velocity,
      'altitude': altitude
  }

In [79]:
def extract_data(frame):
  crop = custom_crop(frame)
  gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
  inverted = cv2.bitwise_not(gray)

  text = pytesseract.image_to_string(inverted)
  return get_props(text)

In [80]:
def await_liftoff():
  ''' Identify the frame at which liftoff occurs '''
  framepos = 0
  interval = 300
  clock = ''
  # Await first mission clock appearance
  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    framepos += interval

    ret, frame = vid.read()
    data = extract_data(frame)

    if data['clock']:
      clock = data['clock']
      break

  # Now wait until the next full second
  interval = 1
  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    framepos += interval

    ret, frame = vid.read()
    data = extract_data(frame)

    if data['clock'] and data['clock'] != clock:
      framepos += data['seconds'] * fps
      return framepos


In [81]:
def process_launch(vid, liftoff_frame):
  interval = 10

  framepos = liftoff_frame
  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    framepos += interval

    ret, frame = vid.read()
    data = extract_data(frame)

    print(data)

In [21]:
url = 'https://www.youtube.com/watch?v=agYuEAkEljw'
get_video(url, 'launches')

Download complete


In [82]:
vidpath = 'launches/launch.mp4'
vid = cv2.VideoCapture(vidpath)
fps = vid.get(cv2.CAP_PROP_FPS)

print('Awaiting liftoff')
liftoff_frame = await_liftoff()

#vid.set(cv2.CAP_PROP_POS_FRAMES, int(liftoff_frame))
#ret, frame = vid.read()
#show(frame)

process_launch(vid, liftoff_frame)

Awaiting liftoff
{'clock': 't+00:00:00', 'seconds': 0, 'velocity': '00', 'altitude': '00'}
{'clock': 't+00:00:00', 'seconds': 0, 'velocity': '00', 'altitude': '00'}
{'clock': 't+00:00:00', 'seconds': 0, 'velocity': '00', 'altitude': '00'}
{'clock': 't+00:00:01', 'seconds': 1, 'velocity': '01', 'altitude': '00'}
{'clock': 't+00:00:01', 'seconds': 1, 'velocity': '7', 'altitude': '0.1'}
{'clock': 't+00:00:01', 'seconds': 1, 'velocity': '9', 'altitude': '0.1'}
{'clock': 't+00:00:02', 'seconds': 2, 'velocity': '02', 'altitude': '00'}
{'clock': '', 'seconds': 0, 'velocity': '02', 'altitude': '00'}
{'clock': '', 'seconds': 0, 'velocity': 0, 'altitude': 0}
{'clock': '', 'seconds': 0, 'velocity': 0, 'altitude': 0}
{'clock': 't+00:00:03', 'seconds': 3, 'velocity': '03', 'altitude': '00'}
{'clock': '', 'seconds': 0, 'velocity': 0, 'altitude': 0}
{'clock': '', 'seconds': 0, 'velocity': 0, 'altitude': 0}
{'clock': '', 'seconds': 0, 'velocity': '04', 'altitude': '00'}
{'clock': '', 'seconds': 0, 've

KeyboardInterrupt: ignored